<a href="https://colab.research.google.com/github/jumanabarghash321/-Rock-Paper-Scissors-Game/blob/main/AI_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1 import nlp,pyTorch,neuralnet libraries

In [3]:
import numpy as np
import random
import json
import nltk
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#2 nlp preprocessing functions(stemming,tokinizing,bag_of_words and remove punctuation)

In [4]:
stemmer = PorterStemmer()

In [5]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

In [6]:
def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1

    return bag


In [ ]:
#3 load my dataset

In [7]:
with open('/content/intents.json', 'r') as f:
    intents = json.load(f)

In [ ]:
#4 apply preprocessing on data

In [8]:
all_words=[]
tags=[]
xy=[]
for intent in intents["intents"]:
  tag=intent['tag']
  tags.append(tag)

  for pattern in intent['patterns']:
    w=tokenize(pattern)
    all_words.extend(w)
    xy.append((w,tag))

In [9]:
ignore_words=['?','.','!']
all_words=[stem(w) for w in all_words if w not in ignore_words]
all_words=sorted(set(all_words))
tags=sorted(set(tags))

print(len(all_words), 'unique stmmed words')


128 unique stmmed words


In [10]:
X_train=[]
y_train=[]

for (pattern_sentence,tag) in  xy:
  bag=bag_of_words(pattern_sentence,all_words)
  X_train.append(bag)
  label=tags.index(tag)
  y_train.append(label)

X_train=np.array(X_train)
y_train=np.array(y_train)


In [ ]:
# 5 create my model (using pytorch to make feedforward neuralnet)

In [11]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()  #instantiate our nn.module
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [12]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # to get by index
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [13]:
num_epochs=1000
batch_size=8
learning_rate=0.001
input_size=len(X_train[0])
hidden_size=8
output_size=len(tags)
print(input_size, output_size)


128 14


In [14]:
dataset=ChatDataset()
train_loader=DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,num_workers=0)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model=NeuralNet(input_size,hidden_size,output_size).to(device)

criterion=nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
#  6 Train the model

In [15]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        # Forward pass
        outputs = model(words)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
print(f'final loss: {loss.item():.4f}')
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

Epoch [100/1000], Loss: 0.7816
Epoch [200/1000], Loss: 0.1019
Epoch [300/1000], Loss: 0.0955
Epoch [400/1000], Loss: 0.0138
Epoch [500/1000], Loss: 0.0019
Epoch [600/1000], Loss: 0.0071
Epoch [700/1000], Loss: 0.0005
Epoch [800/1000], Loss: 0.0004
Epoch [900/1000], Loss: 0.0002
Epoch [1000/1000], Loss: 0.0002
final loss: 0.0002


In [ ]:
# 7 Loading our Saved Model

In [16]:
File="data.pth"
torch.save(data,File)

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('/content/intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()


<ipython-input-17-be973c6eb4cf>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(FILE)


NeuralNet(
  (l1): Linear(in_features=128, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=14, bias=True)
  (relu): ReLU()
)

In [ ]:
#Our Model is Ready. Now we will finally chat with our chat-bot

In [ ]:
#create actual chatbot

In [20]:
bot_name = "chatbot"
print("Let's chat! (type 'quit' to exit)")
while True:
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: hey 
chatbot: Hello! Welcome to our perfume haven. How can I assist you today?
You: What perfumes do you have?
chatbot: We offer a wide selection of perfumes, from luxurious designer scents to niche fragrances. Would you prefer something floral, woody, or perhaps citrus?
You: What perfume do you recommend?
chatbot: For a gift, floral fragrances like Gucci Bloom are popular, but if you know the recipient's preferences, I can make more personalized recommendations. Who is the gift for?
You: Do you take credit cards?
chatbot: We accept VISA, Mastercard, American Express, and PayPal. You can choose whichever is most convenient for you.
You: thank you
chatbot: Happy to help! Enjoy your fragrance.
You: quit
